In [1]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import RidgeClassifier
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression
from adspy_shared_utilities import plot_class_regions_for_classifier_subplot
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from adspy_shared_utilities import plot_decision_tree
from adspy_shared_utilities import plot_class_regions_for_classifier
from adspy_shared_utilities import plot_decision_tree
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from adspy_shared_utilities import plot_decision_tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.ensemble import VotingClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.ensemble import ExtraTreesClassifier
import pickle
import seaborn as sns
from sklearn.inspection import permutation_importance
from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import AdaBoostClassifier
from finta import TA
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import StackingClassifier
from scikeras.wrappers import KerasClassifier
from tensorflow.keras.callbacks import EarlyStopping
from keras import optimizers, metrics

In [2]:
INDICATORS = ['RSI', 'MACD', 'STOCH','ADL', 'ATR', 'MOM', 'MFI', 'ROC', 'OBV', 'CCI', 'EMV', 'VORTEX']
# Df_hourly_merge = pd.read_csv('assets/PCHourly2019202_ActualLabel.csv')
Df_hourly_merge = pd.read_csv('assets/newfeatures/MergeJMJPCHourly2019202_NewsGoogleApi_ActualLabel.csv')

def _exponential_smooth(data, alpha):
    """
    Function that exponentially smooths dataset so values are less 'rigid'
    :param alpha: weight factor to weight recent values more
    """
    
    return data.ewm(alpha=alpha).mean()

Indicatordata = _exponential_smooth(Df_hourly_merge[['Close', 'Open','High','Low','Volume']], 0.65)
def _get_indicator_data(data):
    """
    Function that uses the finta API to calculate technical indicators used as the features
    :return:
    """

    for indicator in INDICATORS:
        ind_data = eval('TA.' + indicator + '(data)')
        if not isinstance(ind_data, pd.DataFrame):
            ind_data = ind_data.to_frame()
        data = data.merge(ind_data, left_index=True, right_index=True)
    data.rename(columns={"14 period EMV.": '14 period EMV'}, inplace=True)

    # Also calculate moving averages for features
    data['ema50'] = data['Close'] / data['Close'].ewm(50).mean()
    data['ema21'] = data['Close'] / data['Close'].ewm(21).mean()
    data['ema15'] = data['Close'] / data['Close'].ewm(14).mean()
    data['ema5'] = data['Close'] / data['Close'].ewm(5).mean()

    # Instead of using the actual volume value (which changes over time), we normalize it with a moving volume average
    data['normVol'] = data['Volume'] / data['Volume'].ewm(5).mean()

    return data

Indicatordatafinal = _get_indicator_data(Indicatordata)

Indicatordatafinal = Indicatordatafinal.drop(['Close', 'Open', 'High', 'Low', 'Volume'], axis = 1)
Df_hourly_merge2 = pd.merge(Df_hourly_merge, Indicatordatafinal, left_index=True, right_index=True)

Df_hourly_merge2 = Df_hourly_merge2.drop(['time','hour','Open Time','_merge','Signal','Position', 'Signal35JMJ', 'Position35JMJ', 'Ignore'], axis = 1)




def binary(value):
  if value > 0:
    return 1
  else:
    return 0

columns = ['favorites', 'retweets', 'number_of_followers', 'following',
       'followers_following_ratio', '2x_retweets_+_favorites', 'polarity',
       'W1 Score', 'Bull_ratio', 'W Score With Bull Ratio', 'Open', 'High',
       'Low', 'Close', 'Volume', 'Quote Asset Volume', 'Number of Trades',
       'TB Base Volume', 'TB Quote Volume', '3MovingAverage', '5MovingAverage',
       'JMJ_3HMoving_averages', 'JMJ_5HMoving_averages', 
       '14 period RSI', 'MACD', 'SIGNAL', '14 period STOCH %K', 'MFV',
       '14 period ATR', 'MOM', '14 period MFI', 'ROC', 'OBV', '20 period CCI',
       '14 period EMV', 'VIm', 'VIp', 'ema50', 'ema21', 'ema15', 'ema5']


for column in columns: 
    Df_hourly_merge2['Binary{}'.format(column)]  = (Df_hourly_merge2[column] - Df_hourly_merge2[column].shift(1)).apply(binary)
    

In [3]:
len(Df_hourly_merge2)

25503

In [4]:
Df_hourly_merge2

favorites  retweets  number_of_followers    following  \
0       6.517857  1.276786         10592.354167  1652.068452   
1       2.685230  0.479419         14341.610169  1852.799031   
2       2.138107  0.670077         21769.074169  2449.731458   
3       1.916462  0.670762         36958.090909  2790.968059   
4       4.339394  0.921212         13345.724242  3208.639394   
...          ...       ...                  ...          ...   
25498   5.783333  0.659524         12129.000000  1975.276190   
25499   3.011905  0.392857         14180.595238  1765.121429   
25500  14.400000  2.407143         16161.992857  2084.061905   
25501  21.570806  3.270153         11254.296296  1814.954248   
25502   7.505967  0.878282         10556.818616  2143.973747   

       followers_following_ratio  2x_retweets_+_favorites  polarity  W1 Score  \
0                      45.605159                 9.071429  0.124241  0.264455   
1                      74.444302                 3.644068  0.067950  0.097316   
2                      68.571009                 3.478261  0.120056  0.144203   
3                     100.500076                 3.257985  0.143717  0.110483   
4                      66.105667                 6.181818  0.136780  0.199699   
...                          ...                      ...       ...       ...   
25498                 108.150291                 7.102381  0.077630  0.078660   
25499                  31.755382                 3.797619  0.095683  0.135134   
25500                 182.260151                19.214286  0.082271  0.107210   
25501                 428.484448                28.111111  0.070485  0.165014   
25502                 140.745063                 9.262530  0.079407  0.135342   

       Bull_ratio  W Score With Bull Ratio  ...  BinaryROC  BinaryOBV  \
0        3.275000                 0.866089  ...          0          0   
1        3.342105                 0.325240  ...          0          0   
2        6.120000                 0.882520  ...          0          0   
3        5.964286                 0.658951  ...          0          0   
4        4.607143                 0.920041  ...          0          0   
...           ...                      ...  ...        ...        ...   
25498    3.022222                 0.237728  ...          0          0   
25499    4.277778                 0.578073  ...          0          0   
25500    3.648649                 0.391172  ...          0          0   
25501    1.887324                 0.311435  ...          0          1   
25502    3.945946                 0.534052  ...          0          0   

       Binary20 period CCI  Binary14 period EMV  BinaryVIm  BinaryVIp  \
0                        0                    0          0          0   
1                        0                    0          0          0   
2                        0                    0          0          0   
3                        0                    0          0          0   
4                        1                    0          0          0   
...                    ...                  ...        ...        ...   
25498                    1                    1          0          1   
25499                    0                    1          0          1   
25500                    0                    0          1          0   
25501                    0                    0          1          0   
25502                    0                    0          1          0   

       Binaryema50  Binaryema21  Binaryema15  Binaryema5  
0                0            0            0           0  
1                1            1            1           1  
2                0            0            0           0  
3                0            0            0           0  
4                0            0            0           0  
...            ...          ...          ...         ...  
25498            0            0            0           0  
25499            0            0        

In [7]:
Df_hourly_merge2.columns.values

array(['favorites', 'retweets', 'number_of_followers', 'following',
       'followers_following_ratio', '2x_retweets_+_favorites', 'polarity',
       'W1 Score', 'Bull_ratio', 'W Score With Bull Ratio', 'Open',
       'High', 'Low', 'Close', 'Volume', 'Quote Asset Volume',
       'Number of Trades', 'TB Base Volume', 'TB Quote Volume',
       '3MovingAverage', '5MovingAverage', 'JMJ_3HMoving_averages',
       'JMJ_5HMoving_averages', 'Actual_Label',
       'Bitcoin_Google_Trend_Score', 'BTC_Google_Trend_Score',
       'Mkt Sentiment', 'Crypto Sentiment',
       'Historically Optimal SMA(s-t)', 'Historically Optimal SMA(l-t)',
       'Historically Optimal WMA(s-t)', 'Historically Optimal WMA(l-t)',
       'Historically Optimal EMA(s-t)', 'Historically Optimal EMA(l-t)',
       'Twitter Hourly Favorites SMA(s-t)',
       'Twitter Hourly Favorites SMA(l-t)',
       'Twitter Hourly Favorites WMA(s-t)',
       'Twitter Hourly Favorites WMA(l-t)',
       'Twitter Hourly Favorites EMA(s-t)',


In [9]:
%%time
data = Df_hourly_merge2
data = data.dropna()

feature_names_JMJ = [
'# of Hourly Trades SMA(l-t)',
'# of Hourly Trades SMA(s-t)',
'# of Hourly Trades WMA(l-t)',
'14 period MFI',
'14 period STOCH %K',
'20 period CCI',
'2x_retweets_+_favorites',
'3MovingAverage',
'BinaryJMJ_3HMoving_averages',
'BinaryJMJ_5HMoving_averages',
'BinaryMFV',
'BinaryOpen',
'BinaryQuote Asset Volume',
'BinaryVolume',
'Bitcoin_Google_Trend_Score',
'BTC_Google_Trend_Score',
'Bull_ratio',
'ema5',
'ema50',
'favorites',
'followers_following_ratio',
'following',
'Historically Optimal EMA(l-t)',
'Historically Optimal SMA(s-t)',
'Historically Optimal WMA(s-t)',
'MFV',
'Mkt Sentiment',
'normVol',
'Number of Trades',
'OBV',
'polarity',
'Quote Asset Volume',
'Quote Asset Volume SMA(l-t)',
'Quote Asset Volume SMA(s-t)',
'retweets',
'ROC',
'TB Base Volume',
'TB Base Volume EMA(l-t)',
'TB Base Volume EMA(s-t)',
'TB Base Volume SMA(l-t)',
'TB Base Volume WMA(l-t)',
'TB Base Volume WMA(s-t)',
'TB Quote Volume',
'TB Quote Volume WMA(s-t)',
'Twitter Hourly 2x Retweets + Favorites EMA(s-t)',
'Twitter Hourly 2x Retweets + Favorites SMA(s-t)',
'Twitter Hourly Bull Ratio EMA(s-t)',
'Twitter Hourly Bull Ratio SMA(l-t)',
'Twitter Hourly Bull Ratio SMA(s-t)',
'Twitter Hourly Bull Ratio WMA(l-t)',
'Twitter Hourly Bull Ratio WMA(s-t)',
'Twitter Hourly Favorites EMA(s-t)',
'Twitter Hourly Favorites WMA(l-t)',
'Twitter Hourly Favorites WMA(s-t)',
'Twitter Hourly Follower Exposure EMA(l-t)',
'Twitter Hourly Follower Exposure EMA(s-t)',
'Twitter Hourly Follower Exposure SMA(s-t)',
'Twitter Hourly Follower Exposure WMA(l-t)' ,
'Twitter Hourly Follower Exposure WMA(s-t)',
'Twitter Hourly Following Exposure EMA(l-t)',
'Twitter Hourly Following Exposure SMA(s-t)',
'Twitter Hourly Following Exposure WMA(s-t)',
'Twitter Hourly Polarity Score EMA(l-t)',
'Twitter Hourly Polarity Score EMA(s-t)',
'Twitter Hourly Polarity Score SMA(l-t)',
'Twitter Hourly Polarity Score SMA(s-t)',
'Twitter Hourly Polarity Score WMA(l-t)',
'Twitter Hourly Polarity Score WMA(s-t)',
'Twitter Hourly Retweets EMA(l-t)',
'Twitter Hourly Retweets SMA(l-t)',
'Twitter Hourly Retweets SMA(s-t)',
'Twitter Hourly Retweets WMA(s-t)',
'Twitter Hourly Weighted Bull Ratio EMA(l-t)',
'Twitter Hourly Weighted Bull Ratio EMA(s-t)',
'Twitter Hourly Weighted Bull Ratio SMA(l-t)',
'Twitter Hourly Weighted Bull Ratio SMA(s-t)',
'Twitter Hourly Weighted Bull Ratio WMA(l-t)',
'Twitter Hourly Weighted Bull Ratio WMA(s-t)',
'Twitter W1 Score EMA(l-t)',
'Twitter W1 Score EMA(s-t)',
'Twitter W1 Score SMA(l-t)',
'Twitter W1 Score SMA(s-t)',
'Twitter W1 Score WMA(l-t)',
'Twitter W1 Score WMA(s-t)',
'VIm',
'Volume',
'W1 Score']

X_JMJ = data[feature_names_JMJ]
y_JMJ = data['Actual_Label']

Wall time: 52.8 ms


In [10]:
len(feature_names_JMJ)

87

## BASE ESTIMATORS

In [11]:
%%time
def _LogisticRegression(X_train, y_train, X_test, y_test):

    """
    Function that uses random forest classifier to train the model
    :return:
    """
    
    # Create a new random forest classifier
    #et = ExtraTreesClassifier()
    lr = LogisticRegression()
    
    # Dictionary of all values we want to test for n_estimators
    params_lr = {
                    'C': [0.01, 0.1, 1, 10, 100], 
                    "penalty":['l2'],
                    'solver': ['lbfgs'],
                    'max_iter':[1000]
        
        
        
                    }
    

    
    # Use gridsearch to test all values for n_estimators
    lr_gs = GridSearchCV(lr, params_lr, cv=5)
    
    # Fit model to training data
    lr_gs.fit(X_train, y_train)
    
    # Save best model
    lr_best = lr_gs.best_estimator_
    
    # Check best n_estimators value
    print(lr_gs.best_params_)
    
    prediction = lr_best.predict(X_test)

    print(classification_report(y_test, prediction))
    print(confusion_matrix(y_test, prediction))
    
    return lr_best
    

Wall time: 0 ns


In [12]:
def _train_AdaBoostClassifier(X_train, y_train, X_test, y_test):

    """
    Function that uses random forest classifier to train the model
    :return:
    """
    
    # Create a new random forest classifier
    abc = AdaBoostClassifier()
    
    # Dictionary of all values we want to test for n_estimators
    params_abc =  {
                'n_estimators': [20, 50, 70, 100],
                'learning_rate' : [0.001, 0.01, 0.1, 0.2]
                }
    
    # Use gridsearch to test all values for n_estimators
    abc_gs = GridSearchCV(abc, params_abc, cv=5)
    
    # Fit model to training data
    abc_gs.fit(X_train, y_train)
    
    # Save best model
    abc_best = abc_gs.best_estimator_
    
    # Check best n_estimators value
    print(abc_gs.best_params_)
    
    prediction = abc_best.predict(X_test)

    print(classification_report(y_test, prediction))
    print(confusion_matrix(y_test, prediction))
    
    return abc_best
    
# abc_model = _train_AdaBoostClassifier(X_train_scaled, y_train, X_test_scaled, y_test)

In [13]:
%%time
def _train_LDA(X_train, y_train, X_test, y_test):

    lda = LinearDiscriminantAnalysis()
    
    
    param_lda = {
                'solver': ['svd', 'lsqr', 'eigen']
                                  
                 }
    
    # Use gridsearch to test all values for n_estimators
    lda_gs = GridSearchCV(lda, param_lda, cv=5)
    
    # Fit model to training data
    lda_gs.fit(X_train, y_train)
              
    # Save best model
    lda_best = lda_gs.best_estimator_
    
    # Check best n_estimators value
    print(lda_gs.best_params_)
    
    prediction = lda_best.predict(X_test)

    print(classification_report(y_test, prediction))
    print(confusion_matrix(y_test, prediction))
    
    return lda_best
    
    
# lda_model = _train_LDA(X_train_scaled, y_train, X_test_scaled, y_test)

Wall time: 0 ns


In [14]:
%%time
def _GradientBoosingClassifier(X_train, y_train, X_test, y_test):


    
    # Create a ne
    GBC = GradientBoostingClassifier()
    
    # Dictionary of all values we want to test for n_estimators
    params_nnclf = {
    "loss":["log_loss"],
    "learning_rate": [0.01, 0.1],
#     "min_samples_split": np.linspace(0.1, 0.5, 12),
#     "min_samples_leaf": np.linspace(0.1, 0.5, 12),
    "max_depth":[3,5],
    "criterion": ["friedman_mse"],
    "subsample":[0.5, 1.0],
    "n_estimators":[100]
    }

    
    # Use gridsearch to test all values for n_estimators
    GBC_gs = GridSearchCV(GBC, params_nnclf, cv=5)
    
    # Fit model to training data
    GBC_gs.fit(X_train, y_train)
#     GBC.fit(X_train, y_train)

    
    # Save best model
    GBC_best = GBC_gs.best_estimator_
    
    # Check best n_estimators value
    print(GBC_gs.best_params_)
    
    prediction = GBC_best.predict(X_test)

    print(classification_report(y_test, prediction))
    print(confusion_matrix(y_test, prediction))
    
    return GBC_best
    
# GBC_model = _GradientBoosingClassifier(X_train_scaled, y_train, X_test_scaled, y_test)

Wall time: 0 ns


## FINAL MODEL SETUP

In [15]:
%%time
tscv = TimeSeriesSplit(n_splits=10)

LTSM_RESULTS = []
abc_RESULTS = []
lda_RESULTS = []
lr_RESULTS =[]
GBC_RESULTS = []
    
for train_index, test_index in tscv.split(X_JMJ):
    print(train_index, test_index)
    
    X_train, X_test = X_JMJ.iloc[train_index], X_JMJ.iloc[test_index]
    y_train, y_test = y_JMJ.iloc[train_index], y_JMJ.iloc[test_index]

    scaler = MinMaxScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    #Setting up the Base Estimators
    lr_model = _LogisticRegression(X_train_scaled, y_train, X_test_scaled, y_test)
    abc_model = _train_AdaBoostClassifier(X_train_scaled, y_train,  X_test_scaled, y_test)
    lda_model = _train_LDA(X_train_scaled, y_train, X_test_scaled, y_test)
    GBC_model = _GradientBoosingClassifier(X_train_scaled, y_train, X_test_scaled, y_test)
    
    # Define the stacking classifier
    stacking_classifier = StackingClassifier(
        estimators=[('lr', lr_model),('abc', abc_model),('lda', lda_model), ('GBC',GBC_model)], 
        final_estimator=lr_model,
        cv=5,
        passthrough=True,
        stack_method='predict_proba'
    )
    
    # Train the stacking classifier on the training data
    stacking_classifier.fit(X_train_scaled, y_train)

    
    # Make predictions
    y_pred = stacking_classifier.predict(X_test_scaled)
    testy_pred = (y_pred >= 0.5).astype(int)
    
    abc_prediction = abc_model.predict(X_test_scaled)
    lda_prediction = lda_model.predict(X_test_scaled)
    lr_prediction = lr_model.predict(X_test_scaled)
    GBC_prediction = GBC_model.predict(X_test_scaled)

    
    abc_accuracy = accuracy_score(y_test.values, abc_prediction)
    lda_accuracy = accuracy_score(y_test.values, lda_prediction)
    lr_accuracy = accuracy_score(y_test.values, lr_prediction)
    GBC_accuracy = accuracy_score(y_test.values, GBC_prediction)
    LTSMaccurary = accuracy_score (y_test, testy_pred)   
   


    abc_RESULTS.append(abc_accuracy)
    lda_RESULTS.append(lda_accuracy)
    lr_RESULTS.append(lr_accuracy)
    GBC_RESULTS.append(GBC_accuracy)
    LTSM_RESULTS.append(LTSMaccurary)
        
    # Print classification report
    print(classification_report(y_test, testy_pred))
    print(confusion_matrix(y_test, testy_pred))
    
    print('ABC Accuracy = ' + str(abc_accuracy),', LDA Accuracy = ' + str(lda_accuracy),', LR Accuracy = ' + str(lr_accuracy),', GBC Accuracy = ' + str(GBC_accuracy),', LTSM Accuracy = ' + str(LTSMaccurary))#,GBC_accuracy, ensemble_accuracy)    
    print(' ')
    
    
    

    
print(' ')
print('ABC Accuracy = ' + str( sum(abc_RESULTS) / len(abc_RESULTS)))
print('LDA Accuracy = ' + str( sum(lda_RESULTS) / len(lda_RESULTS)))
print('LR Accuracy = ' + str( sum(lr_RESULTS) / len(lr_RESULTS)))
print('GBC Accuracy = ' + str( sum(GBC_RESULTS) / len(GBC_RESULTS)))
print('LTSM Accuracy = ' + str( sum(LTSM_RESULTS) / len(LTSM_RESULTS)))
print(' ')

[   0    1    2 ... 2316 2317 2318] [2319 2320 2321 ... 4633 4634 4635]
{'C': 0.1, 'max_iter': 1000, 'penalty': 'l2', 'solver': 'lbfgs'}
              precision    recall  f1-score   support

         0.0       0.54      0.40      0.46      1084
         1.0       0.57      0.69      0.63      1233

    accuracy                           0.56      2317
   macro avg       0.55      0.55      0.54      2317
weighted avg       0.55      0.56      0.55      2317

[[436 648]
 [377 856]]
{'learning_rate': 0.001, 'n_estimators': 20}
              precision    recall  f1-score   support

         0.0       0.51      0.67      0.58      1084
         1.0       0.60      0.43      0.50      1233

    accuracy                           0.54      2317
   macro avg       0.55      0.55      0.54      2317
weighted avg       0.56      0.54      0.54      2317

[[724 360]
 [697 536]]


C:\Users\mario\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:378: FitFailedWarning: 
1 fits failed out of a total of 15.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
1 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\mario\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\mario\anaconda3\lib\site-packages\sklearn\discriminant_analysis.py", line 631, in fit
    self._solve_eigen(
  File "C:\Users\mario\anaconda3\lib\site-packages\sklearn\discriminant_analysis.py", line 463, in _solve_eigen
    evals, evecs = linalg.eigh(Sb, Sw)
  File "C:\Users\mario\anaconda3\li

{'solver': 'svd'}
              precision    recall  f1-score   support

         0.0       0.58      0.05      0.09      1084
         1.0       0.54      0.97      0.69      1233

    accuracy                           0.54      2317
   macro avg       0.56      0.51      0.39      2317
weighted avg       0.56      0.54      0.41      2317

[[  52 1032]
 [  37 1196]]
{'criterion': 'friedman_mse', 'learning_rate': 0.01, 'loss': 'log_loss', 'max_depth': 3, 'n_estimators': 100, 'subsample': 0.5}
              precision    recall  f1-score   support

         0.0       0.53      0.45      0.49      1084
         1.0       0.57      0.65      0.61      1233

    accuracy                           0.56      2317
   macro avg       0.55      0.55      0.55      2317
weighted avg       0.55      0.56      0.55      2317

[[492 592]
 [433 800]]
              precision    recall  f1-score   support

         0.0       0.53      0.20      0.29      1084
         1.0       0.54      0.84      0.

C:\Users\mario\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:378: FitFailedWarning: 
4 fits failed out of a total of 15.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
4 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\mario\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\mario\anaconda3\lib\site-packages\sklearn\discriminant_analysis.py", line 631, in fit
    self._solve_eigen(
  File "C:\Users\mario\anaconda3\lib\site-packages\sklearn\discriminant_analysis.py", line 463, in _solve_eigen
    evals, evecs = linalg.eigh(Sb, Sw)
  File "C:\Users\mario\anaconda3\li

{'solver': 'lsqr'}
              precision    recall  f1-score   support

         0.0       0.62      0.04      0.08      1178
         1.0       0.50      0.97      0.66      1139

    accuracy                           0.50      2317
   macro avg       0.56      0.51      0.37      2317
weighted avg       0.56      0.50      0.36      2317

[[  50 1128]
 [  30 1109]]
{'criterion': 'friedman_mse', 'learning_rate': 0.01, 'loss': 'log_loss', 'max_depth': 3, 'n_estimators': 100, 'subsample': 1.0}
              precision    recall  f1-score   support

         0.0       0.61      0.30      0.40      1178
         1.0       0.53      0.80      0.64      1139

    accuracy                           0.55      2317
   macro avg       0.57      0.55      0.52      2317
weighted avg       0.57      0.55      0.52      2317

[[350 828]
 [223 916]]
              precision    recall  f1-score   support

         0.0       0.61      0.32      0.42      1178
         1.0       0.53      0.79      0

...

{'solver': 'svd'}
              precision    recall  f1-score   support

         0.0       0.54      0.54      0.54      1151
         1.0       0.55      0.54      0.54      1166

    accuracy                           0.54      2317
   macro avg       0.54      0.54      0.54      2317
weighted avg       0.54      0.54      0.54      2317

[[625 526]
 [534 632]]
{'criterion': 'friedman_mse', 'learning_rate': 0.01, 'loss': 'log_loss', 'max_depth': 3, 'n_estimators': 100, 'subsample': 0.5}
              precision    recall  f1-score   support

         0.0       0.55      0.38      0.45      1151
         1.0       0.53      0.69      0.60      1166

    accuracy                           0.54      2317
   macro avg       0.54      0.54      0.53      2317
weighted avg       0.54      0.54      0.53      2317

[[442 709]
 [359 807]]
              precision    recall  f1-score   support

         0.0       0.53      0.45      0.49      1151
         1.0       0.53      0.62      0.57  

C:\Users\mario\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\mario\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_

{'C': 0.01, 'max_iter': 1000, 'penalty': 'l2', 'solver': 'lbfgs'}
              precision    recall  f1-score   support

         0.0       0.56      0.55      0.55      1166
         1.0       0.55      0.55      0.55      1151

    accuracy                           0.55      2317
   macro avg       0.55      0.55      0.55      2317
weighted avg       0.55      0.55      0.55      2317

[[646 520]
 [517 634]]
{'learning_rate': 0.001, 'n_estimators': 20}
              precision    recall  f1-score   support

         0.0       0.55      0.57      0.56      1166
         1.0       0.55      0.53      0.54      1151

    accuracy                           0.55      2317
   macro avg       0.55      0.55      0.55      2317
weighted avg       0.55      0.55      0.55      2317

[[662 504]
 [542 609]]


C:\Users\mario\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:378: FitFailedWarning: 
3 fits failed out of a total of 15.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
3 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\mario\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\mario\anaconda3\lib\site-packages\sklearn\discriminant_analysis.py", line 631, in fit
    self._solve_eigen(
  File "C:\Users\mario\anaconda3\lib\site-packages\sklearn\discriminant_analysis.py", line 463, in _solve_eigen
    evals, evecs = linalg.eigh(Sb, Sw)
  File "C:\Users\mario\anaconda3\li

{'solver': 'svd'}
              precision    recall  f1-score   support

         0.0       0.55      0.49      0.52      1166
         1.0       0.54      0.60      0.57      1151

    accuracy                           0.55      2317
   macro avg       0.55      0.55      0.54      2317
weighted avg       0.55      0.55      0.54      2317

[[573 593]
 [461 690]]
{'criterion': 'friedman_mse', 'learning_rate': 0.01, 'loss': 'log_loss', 'max_depth': 3, 'n_estimators': 100, 'subsample': 1.0}
              precision    recall  f1-score   support

         0.0       0.56      0.46      0.51      1166
         1.0       0.54      0.64      0.58      1151

    accuracy                           0.55      2317
   macro avg       0.55      0.55      0.55      2317
weighted avg       0.55      0.55      0.55      2317

[[539 627]
 [419 732]]
              precision    recall  f1-score   support

         0.0       0.56      0.55      0.55      1166
         1.0       0.55      0.55      0.55  

In [16]:
stacking_classifier

StackingClassifier(cv=5,
                   estimators=[('lr',
                                LogisticRegression(C=0.01, max_iter=1000)),
                               ('abc',
                                AdaBoostClassifier(learning_rate=0.001,
                                                   n_estimators=20)),
                               ('lda', LinearDiscriminantAnalysis()),
                               ('GBC',
                                GradientBoostingClassifier(learning_rate=0.01))],
                   final_estimator=LogisticRegression(C=0.01, max_iter=1000),
                   passthrough=True, stack_method='predict_proba')

In [17]:
print(stacking_classifier)

StackingClassifier(cv=5,
                   estimators=[('lr',
                                LogisticRegression(C=0.01, max_iter=1000)),
                               ('abc',
                                AdaBoostClassifier(learning_rate=0.001,
                                                   n_estimators=20)),
                               ('lda', LinearDiscriminantAnalysis()),
                               ('GBC',
                                GradientBoostingClassifier(learning_rate=0.01))],
                   final_estimator=LogisticRegression(C=0.01, max_iter=1000),
                   passthrough=True, stack_method='predict_proba')


#### SAVING MODEL

In [20]:
import pickle
#saving model
pkl_filename = 'model/Priority_Feature_Stacking_LR_model_JT2019to2022TEST_news_google.pkl'
with open(pkl_filename, 'wb') as file:
    pickle.dump(stacking_classifier, file)

#### LOADING MODEL:

In [2]:
import pickle 
pkl_filename = 'model/Priority_Feature_Stacking_LR_model_JT2019to2022TEST_news_google.pkl'
with open(pkl_filename, 'rb') as file: 
    Model2 = pickle.load(file)  

# TESTING RESULTS

### Testing with Whole Data

In [ ]:
# Models = [Model1,abc_model, rc_model, lda_model,lr_model,GBC_model]

In [21]:
INDICATORS = ['RSI', 'MACD', 'STOCH','ADL', 'ATR', 'MOM', 'MFI', 'ROC', 'OBV', 'CCI', 'EMV', 'VORTEX']

January = 'Test data/PCHourly202201.csv'
February= 'Test data/PCHourly202202.csv'
March = 'Test data/PCHourly202203.csv'
April = 'Test data/PCHourly202204.csv'
May= 'Test data/PCHourly202205.csv'
June = 'Test data/PCHourly202206.csv'
July = 'Test data/PCHourly202207.csv'
August= 'Test data/PCHourly202208.csv'
September = 'Test data/PCHourly202209.csv'
October = 'Test data/PCHourly202210.csv'

testdata2022 ='assets/newfeatures/TEST_MergeJMJPCHourly2019202_NewsGoogleApi_ActualLabel.csv'

files= [testdata2022]

def _exponential_smooth(data, alpha):
    """
    Function that exponentially smooths dataset so values are less 'rigid'
    :param alpha: weight factor to weight recent values more
    """
    
    return data.ewm(alpha=alpha).mean()

def _get_indicator_data(data):
    """
    Function that uses the finta API to calculate technical indicators used as the features
    :return:
    """

    for indicator in INDICATORS:
        ind_data = eval('TA.' + indicator + '(data)')
        if not isinstance(ind_data, pd.DataFrame):
            ind_data = ind_data.to_frame()
        data = data.merge(ind_data, left_index=True, right_index=True)
    data.rename(columns={"14 period EMV.": '14 period EMV'}, inplace=True)

    # Also calculate moving averages for features
    data['ema50'] = data['Close'] / data['Close'].ewm(50).mean()
    data['ema21'] = data['Close'] / data['Close'].ewm(21).mean()
    data['ema15'] = data['Close'] / data['Close'].ewm(14).mean()
    data['ema5'] = data['Close'] / data['Close'].ewm(5).mean()

    # Instead of using the actual volume value (which changes over time), we normalize it with a moving volume average
    data['normVol'] = data['Volume'] / data['Volume'].ewm(5).mean()

    return data


for file,x in zip(files, ['testdata2022']):
    PredictDF= pd.read_csv(file)
    prediction = (PredictDF.shift(-1)['Close'] >= PredictDF['Close'])
    prediction = prediction.iloc[:-1]
    PredictDF['Actual_Label'] = prediction.astype(int)
    PredictDF= PredictDF.dropna()
    
    Indicatordata = _exponential_smooth(PredictDF[['Close', 'Open','High','Low','Volume']], 0.65)

    Indicatordatafinal = _get_indicator_data(Indicatordata)

    Indicatordatafinal = Indicatordatafinal.drop(['Close', 'Open', 'High', 'Low', 'Volume'], axis = 1)
    PredictDF = pd.merge(PredictDF, Indicatordatafinal, left_index=True, right_index=True)
    PredictDF = PredictDF.drop(['time','hour','Open Time','_merge','Signal','Position', 'Signal35JMJ', 'Position35JMJ', 'Ignore'], axis = 1)

    
    
    
    
    columns = ['favorites', 'retweets', 'number_of_followers', 'following',
           'followers_following_ratio', '2x_retweets_+_favorites', 'polarity',
           'W1 Score', 'Bull_ratio', 'W Score With Bull Ratio', 'Open', 'High',
           'Low', 'Close', 'Volume', 'Quote Asset Volume', 'Number of Trades',
           'TB Base Volume', 'TB Quote Volume', '3MovingAverage', '5MovingAverage',
           'JMJ_3HMoving_averages', 'JMJ_5HMoving_averages', 
           '14 period RSI', 'MACD', 'SIGNAL', '14 period STOCH %K', 'MFV',
           '14 period ATR', 'MOM', '14 period MFI', 'ROC', 'OBV', '20 period CCI',
           '14 period EMV', 'VIm', 'VIp', 'ema50', 'ema21', 'ema15', 'ema5']




    for column in columns: 
        PredictDF['Binary{}'.format(column)]  = (PredictDF[column] - PredictDF[column].shift(1)).apply(binary)
    

    PredictDF = PredictDF.dropna()

    
    feature_names_JMJ = [
    '# of Hourly Trades SMA(l-t)',
    '# of Hourly Trades SMA(s-t)',
    '# of Hourly Trades WMA(l-t)',
    '14 period MFI',
    '14 period STOCH %K',
    '20 period CCI',
    '2x_retweets_+_favorites',
    '3MovingAverage',
    'BinaryJMJ_3HMoving_averages',
    'BinaryJMJ_5HMoving_averages',
    'BinaryMFV',
    'BinaryOpen',
    'BinaryQuote Asset Volume',
    'BinaryVolume',
    'Bitcoin_Google_Trend_Score',
    'BTC_Google_Trend_Score',
    'Bull_ratio',
    'ema5',
    'ema50',
    'favorites',
    'followers_following_ratio',
    'following',
    'Historically Optimal EMA(l-t)',
    'Historically Optimal SMA(s-t)',
    'Historically Optimal WMA(s-t)',
    'MFV',
    'Mkt Sentiment',
    'normVol',
    'Number of Trades',
    'OBV',
    'polarity',
    'Quote Asset Volume',
    'Quote Asset Volume SMA(l-t)',
    'Quote Asset Volume SMA(s-t)',
    'retweets',
    'ROC',
    'TB Base Volume',
    'TB Base Volume EMA(l-t)',
    'TB Base Volume EMA(s-t)',
    'TB Base Volume SMA(l-t)',
    'TB Base Volume WMA(l-t)',
    'TB Base Volume WMA(s-t)',
    'TB Quote Volume',
    'TB Quote Volume WMA(s-t)',
    'Twitter Hourly 2x Retweets + Favorites EMA(s-t)',
    'Twitter Hourly 2x Retweets + Favorites SMA(s-t)',
    'Twitter Hourly Bull Ratio EMA(s-t)',
    'Twitter Hourly Bull Ratio SMA(l-t)',
    'Twitter Hourly Bull Ratio SMA(s-t)',
    'Twitter Hourly Bull Ratio WMA(l-t)',
    'Twitter Hourly Bull Ratio WMA(s-t)',
    'Twitter Hourly Favorites EMA(s-t)',
    'Twitter Hourly Favorites WMA(l-t)',
    'Twitter Hourly Favorites WMA(s-t)',
    'Twitter Hourly Follower Exposure EMA(l-t)',
    'Twitter Hourly Follower Exposure EMA(s-t)',
    'Twitter Hourly Follower Exposure SMA(s-t)',
    'Twitter Hourly Follower Exposure WMA(l-t)' ,
    'Twitter Hourly Follower Exposure WMA(s-t)',
    'Twitter Hourly Following Exposure EMA(l-t)',
    'Twitter Hourly Following Exposure SMA(s-t)',
    'Twitter Hourly Following Exposure WMA(s-t)',
    'Twitter Hourly Polarity Score EMA(l-t)',
    'Twitter Hourly Polarity Score EMA(s-t)',
    'Twitter Hourly Polarity Score SMA(l-t)',
    'Twitter Hourly Polarity Score SMA(s-t)',
    'Twitter Hourly Polarity Score WMA(l-t)',
    'Twitter Hourly Polarity Score WMA(s-t)',
    'Twitter Hourly Retweets EMA(l-t)',
    'Twitter Hourly Retweets SMA(l-t)',
    'Twitter Hourly Retweets SMA(s-t)',
    'Twitter Hourly Retweets WMA(s-t)',
    'Twitter Hourly Weighted Bull Ratio EMA(l-t)',
    'Twitter Hourly Weighted Bull Ratio EMA(s-t)',
    'Twitter Hourly Weighted Bull Ratio SMA(l-t)',
    'Twitter Hourly Weighted Bull Ratio SMA(s-t)',
    'Twitter Hourly Weighted Bull Ratio WMA(l-t)',
    'Twitter Hourly Weighted Bull Ratio WMA(s-t)',
    'Twitter W1 Score EMA(l-t)',
    'Twitter W1 Score EMA(s-t)',
    'Twitter W1 Score SMA(l-t)',
    'Twitter W1 Score SMA(s-t)',
    'Twitter W1 Score WMA(l-t)',
    'Twitter W1 Score WMA(s-t)',
    'VIm',
    'Volume',
    'W1 Score']


    X_test = PredictDF[feature_names_JMJ]
    y_test = PredictDF['Actual_Label']

    scaler = MinMaxScaler()
    X_train_scaled = scaler.fit_transform(X_test)
    X_test_scaled = scaler.transform(X_test)

    predict_y=  stacking_classifier.predict(X_test_scaled)
    PredictDF['Predicted_Label']= predict_y

    print(classification_report(y_test, predict_y))   
    
    # confusion_mc = confusion_matrix(y_test_mc, svm_predicted_mc)
    confusion_mc = confusion_matrix(PredictDF['Actual_Label'], PredictDF['Predicted_Label'])
    df_cm = pd.DataFrame(confusion_mc, 
                         index = [i for i in range(0,2)], columns = [i for i in range(0,2)])


    PredictDF['ValueActual'] =0
    PredictDF['BTCValueActual'] =0
    if PredictDF.loc[PredictDF.index[0],'Actual_Label']==0.0:
        PredictDF.loc[PredictDF.index[0],'ValueActual']=1000
    else:
        PredictDF.loc[PredictDF.index[0],'BTCValueActual']=round((1000/PredictDF.loc[PredictDF.index[0],'Close']),5)


    for current in range(1, len(PredictDF.index)):
        previous = current - 1

        if PredictDF.loc[PredictDF.index[current],'Actual_Label']==0 and PredictDF.loc[PredictDF.index[previous],'BTCValueActual']==0:
            PredictDF.loc[PredictDF.index[current],'ValueActual']=PredictDF.loc[PredictDF.index[previous],'ValueActual']
        elif PredictDF.loc[PredictDF.index[current],'Actual_Label']==1 and PredictDF.loc[PredictDF.index[previous],'ValueActual'] ==0 :
            PredictDF.loc[PredictDF.index[current],'ValueActual']= 0
            PredictDF.loc[PredictDF.index[current],'BTCValueActual']=round(PredictDF.loc[PredictDF.index[previous],'BTCValueActual'],3)
        elif PredictDF.loc[PredictDF.index[current],'Actual_Label']==1:
            PredictDF.loc[PredictDF.index[current],'BTCValueActual'] = round((PredictDF.loc[PredictDF.index[previous],'ValueActual']/PredictDF.loc[PredictDF.index[current],'Close']),5)
        else:
            PredictDF.loc[PredictDF.index[current],'ValueActual'] = round((PredictDF.loc[PredictDF.index[previous],'BTCValueActual'] *PredictDF.loc[PredictDF.index[current],'Close']),3)


    PredictDF['ValuePredicted'] =0
    PredictDF['BTCValuePredicted'] =0
    if PredictDF.loc[PredictDF.index[0],'Predicted_Label']==0.0:
        PredictDF.loc[PredictDF.index[0],'ValuePredicted']=1000
    else:
        PredictDF.loc[PredictDF.index[0],'BTCValuePredicted']=round((1000/PredictDF.loc[PredictDF.index[0],'Close']),5)


    for current in range(1, len(PredictDF.index)):
        previous = current - 1

        if PredictDF.loc[PredictDF.index[current],'Predicted_Label']==0 and PredictDF.loc[PredictDF.index[previous],'BTCValuePredicted']==0:
            PredictDF.loc[PredictDF.index[current],'ValuePredicted']=PredictDF.loc[PredictDF.index[previous],'ValuePredicted']
        elif PredictDF.loc[PredictDF.index[current],'Predicted_Label']==1 and PredictDF.loc[PredictDF.index[previous],'ValuePredicted'] ==0 :
            PredictDF.loc[PredictDF.index[current],'ValuePredicted']= 0
            PredictDF.loc[PredictDF.index[current],'BTCValuePredicted']=round(PredictDF.loc[PredictDF.index[previous],'BTCValuePredicted'],3)
        elif PredictDF.loc[PredictDF.index[current],'Predicted_Label']==1:
            PredictDF.loc[PredictDF.index[current],'BTCValuePredicted'] = round((PredictDF.loc[PredictDF.index[previous],'ValuePredicted']/PredictDF.loc[PredictDF.index[current],'Close']),5)
        else:
            PredictDF.loc[PredictDF.index[current],'ValuePredicted'] = round((PredictDF.loc[PredictDF.index[previous],'BTCValuePredicted'] *PredictDF.loc[PredictDF.index[current],'Close']),3)
    df = PredictDF.mask(PredictDF==0).ffill().iloc[[-1]]
    LastPredictvalue = df['ValuePredicted'].values[0]
    LastPredictBTC = df['BTCValuePredicted'].values[0]
    # LastPredictvalue= PredictDF['ValuePredicted'][PredictDF['ValuePredicted'].to_numpy().nonzero()[0][-1]+1]
    # LastPredictBTC= PredictDF['BTCValuePredicted'][PredictDF['BTCValuePredicted'].to_numpy().nonzero()[0][-1]+1]

    #     print('Total USD Value and BTC Benchmark Actual Label: ${:,.2f} and {:,.2f} BTC'.format (max(PredictDF['ValueActual']),max(PredictDF['BTCValueActual'])))#, )       
    #     print('Total USD Value and BTC Benchmark Actual Label: ${:,.2f} and {:,.2f} BTC'.format (LastPredictvalue,LastPredictBTC))#, )       
    #     print (' ')


    plt.figure(figsize=(2,2))
    sns.heatmap(df_cm, annot=True, fmt='g')
    plt.title('Assessment for: {}\nAccuracy:{:.3f}\nPrecision:{:.3f}\nRecall:{:.3f}\nF1:{:.3f}\nBenchmark USD Value and BTC  ${:,.2f} and {:,.2f}\nPredicted USD Value and BTC ${:,.2f} and {:,.2f}'.format(x, accuracy_score(PredictDF['Actual_Label'],PredictDF['Predicted_Label']),
                                                                        precision_score(PredictDF['Actual_Label'],PredictDF['Predicted_Label']),
                                                                                    recall_score(PredictDF['Actual_Label'], PredictDF['Predicted_Label']),
                                                                                    f1_score(PredictDF['Actual_Label'], PredictDF['Predicted_Label']),
                                                                                     max(PredictDF['ValueActual']),max(PredictDF['BTCValueActual']),
                                                                                     LastPredictvalue,LastPredictBTC
                                                                                    ))
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

              precision    recall  f1-score   support

         0.0       0.56      0.47      0.51      3625
         1.0       0.54      0.63      0.58      3629

    accuracy                           0.55      7254
   macro avg       0.55      0.55      0.54      7254
weighted avg       0.55      0.55      0.54      7254



## Trying with Thesholds

In [22]:
INDICATORS = ['RSI', 'MACD', 'STOCH','ADL', 'ATR', 'MOM', 'MFI', 'ROC', 'OBV', 'CCI', 'EMV', 'VORTEX']

testdata2022 ='assets/newfeatures/TEST_MergeJMJPCHourly2019202_NewsGoogleApi_ActualLabel.csv'

files= [testdata2022]

def _exponential_smooth(data, alpha):
    """
    Function that exponentially smooths dataset so values are less 'rigid'
    :param alpha: weight factor to weight recent values more
    """
    
    return data.ewm(alpha=alpha).mean()

def _get_indicator_data(data):
    """
    Function that uses the finta API to calculate technical indicators used as the features
    :return:
    """

    for indicator in INDICATORS:
        ind_data = eval('TA.' + indicator + '(data)')
        if not isinstance(ind_data, pd.DataFrame):
            ind_data = ind_data.to_frame()
        data = data.merge(ind_data, left_index=True, right_index=True)
    data.rename(columns={"14 period EMV.": '14 period EMV'}, inplace=True)

    # Also calculate moving averages for features
    data['ema50'] = data['Close'] / data['Close'].ewm(50).mean()
    data['ema21'] = data['Close'] / data['Close'].ewm(21).mean()
    data['ema15'] = data['Close'] / data['Close'].ewm(14).mean()
    data['ema5'] = data['Close'] / data['Close'].ewm(5).mean()

    # Instead of using the actual volume value (which changes over time), we normalize it with a moving volume average
    data['normVol'] = data['Volume'] / data['Volume'].ewm(5).mean()

    return data

UpperThreshold = 0.6
LowerThresshold = 0.4

for file,x in zip(files, ['testdata2022']):
    PredictDF= pd.read_csv(file)
    prediction = (PredictDF.shift(-1)['Close'] >= PredictDF['Close'])
    prediction = prediction.iloc[:-1]
    PredictDF['Actual_Label'] = prediction.astype(int)
    PredictDF= PredictDF.dropna()
    
    Indicatordata = _exponential_smooth(PredictDF[['Close', 'Open','High','Low','Volume']], 0.65)

    Indicatordatafinal = _get_indicator_data(Indicatordata)

    Indicatordatafinal = Indicatordatafinal.drop(['Close', 'Open', 'High', 'Low', 'Volume'], axis = 1)
    PredictDF = pd.merge(PredictDF, Indicatordatafinal, left_index=True, right_index=True)
    PredictDF = PredictDF.drop(['time','hour','Open Time','_merge','Signal','Position', 'Signal35JMJ', 'Position35JMJ', 'Ignore'], axis = 1)

    
    
    
    
    columns = ['favorites', 'retweets', 'number_of_followers', 'following',
           'followers_following_ratio', '2x_retweets_+_favorites', 'polarity',
           'W1 Score', 'Bull_ratio', 'W Score With Bull Ratio', 'Open', 'High',
           'Low', 'Close', 'Volume', 'Quote Asset Volume', 'Number of Trades',
           'TB Base Volume', 'TB Quote Volume', '3MovingAverage', '5MovingAverage',
           'JMJ_3HMoving_averages', 'JMJ_5HMoving_averages', 
           '14 period RSI', 'MACD', 'SIGNAL', '14 period STOCH %K', 'MFV',
           '14 period ATR', 'MOM', '14 period MFI', 'ROC', 'OBV', '20 period CCI',
           '14 period EMV', 'VIm', 'VIp', 'ema50', 'ema21', 'ema15', 'ema5']




    for column in columns: 
        PredictDF['Binary{}'.format(column)]  = (PredictDF[column] - PredictDF[column].shift(1)).apply(binary)
    

    PredictDF = PredictDF.dropna()
#     feature_names_JMJ = ['favorites', 'retweets', 'number_of_followers', 'following',
#            'followers_following_ratio', '2x_retweets_+_favorites', 'polarity',
#            'W1 Score', 'Bull_ratio', 'W Score With Bull Ratio', 'Open', 'High',
#            'Low', 'Close', 'Volume', 'Quote Asset Volume', 'Number of Trades',
#            'TB Base Volume', 'TB Quote Volume', '3MovingAverage', '5MovingAverage',
#            'JMJ_3HMoving_averages', 'JMJ_5HMoving_averages',
#            '14 period RSI', 'MACD', 'SIGNAL', '14 period STOCH %K', 'MFV',
#            '14 period ATR', 'MOM', '14 period MFI', 'ROC', 'OBV', '20 period CCI',
#            '14 period EMV', 'VIm', 'VIp', 'ema50', 'ema21', 'ema15', 'ema5',
#            'normVol', 'Binaryfavorites', 'Binaryretweets',
#            'Binarynumber_of_followers', 'Binaryfollowing',
#            'Binaryfollowers_following_ratio', 'Binary2x_retweets_+_favorites',
#            'Binarypolarity', 'BinaryW1 Score', 'BinaryBull_ratio',
#            'BinaryW Score With Bull Ratio', 'BinaryOpen', 'BinaryHigh',
#            'BinaryLow', 'BinaryClose', 'BinaryVolume', 'BinaryQuote Asset Volume',
#            'BinaryNumber of Trades', 'BinaryTB Base Volume',
#            'BinaryTB Quote Volume', 'Binary3MovingAverage', 'Binary5MovingAverage',
#            'BinaryJMJ_3HMoving_averages', 'BinaryJMJ_5HMoving_averages']
    
    
    feature_names_JMJ = [
    '# of Hourly Trades SMA(l-t)',
    '# of Hourly Trades SMA(s-t)',
    '# of Hourly Trades WMA(l-t)',
    '14 period MFI',
    '14 period STOCH %K',
    '20 period CCI',
    '2x_retweets_+_favorites',
    '3MovingAverage',
    'BinaryJMJ_3HMoving_averages',
    'BinaryJMJ_5HMoving_averages',
    'BinaryMFV',
    'BinaryOpen',
    'BinaryQuote Asset Volume',
    'BinaryVolume',
    'Bitcoin_Google_Trend_Score',
    'BTC_Google_Trend_Score',
    'Bull_ratio',
    'ema5',
    'ema50',
    'favorites',
    'followers_following_ratio',
    'following',
    'Historically Optimal EMA(l-t)',
    'Historically Optimal SMA(s-t)',
    'Historically Optimal WMA(s-t)',
    'MFV',
    'Mkt Sentiment',
    'normVol',
    'Number of Trades',
    'OBV',
    'polarity',
    'Quote Asset Volume',
    'Quote Asset Volume SMA(l-t)',
    'Quote Asset Volume SMA(s-t)',
    'retweets',
    'ROC',
    'TB Base Volume',
    'TB Base Volume EMA(l-t)',
    'TB Base Volume EMA(s-t)',
    'TB Base Volume SMA(l-t)',
    'TB Base Volume WMA(l-t)',
    'TB Base Volume WMA(s-t)',
    'TB Quote Volume',
    'TB Quote Volume WMA(s-t)',
    'Twitter Hourly 2x Retweets + Favorites EMA(s-t)',
    'Twitter Hourly 2x Retweets + Favorites SMA(s-t)',
    'Twitter Hourly Bull Ratio EMA(s-t)',
    'Twitter Hourly Bull Ratio SMA(l-t)',
    'Twitter Hourly Bull Ratio SMA(s-t)',
    'Twitter Hourly Bull Ratio WMA(l-t)',
    'Twitter Hourly Bull Ratio WMA(s-t)',
    'Twitter Hourly Favorites EMA(s-t)',
    'Twitter Hourly Favorites WMA(l-t)',
    'Twitter Hourly Favorites WMA(s-t)',
    'Twitter Hourly Follower Exposure EMA(l-t)',
    'Twitter Hourly Follower Exposure EMA(s-t)',
    'Twitter Hourly Follower Exposure SMA(s-t)',
    'Twitter Hourly Follower Exposure WMA(l-t)' ,
    'Twitter Hourly Follower Exposure WMA(s-t)',
    'Twitter Hourly Following Exposure EMA(l-t)',
    'Twitter Hourly Following Exposure SMA(s-t)',
    'Twitter Hourly Following Exposure WMA(s-t)',
    'Twitter Hourly Polarity Score EMA(l-t)',
    'Twitter Hourly Polarity Score EMA(s-t)',
    'Twitter Hourly Polarity Score SMA(l-t)',
    'Twitter Hourly Polarity Score SMA(s-t)',
    'Twitter Hourly Polarity Score WMA(l-t)',
    'Twitter Hourly Polarity Score WMA(s-t)',
    'Twitter Hourly Retweets EMA(l-t)',
    'Twitter Hourly Retweets SMA(l-t)',
    'Twitter Hourly Retweets SMA(s-t)',
    'Twitter Hourly Retweets WMA(s-t)',
    'Twitter Hourly Weighted Bull Ratio EMA(l-t)',
    'Twitter Hourly Weighted Bull Ratio EMA(s-t)',
    'Twitter Hourly Weighted Bull Ratio SMA(l-t)',
    'Twitter Hourly Weighted Bull Ratio SMA(s-t)',
    'Twitter Hourly Weighted Bull Ratio WMA(l-t)',
    'Twitter Hourly Weighted Bull Ratio WMA(s-t)',
    'Twitter W1 Score EMA(l-t)',
    'Twitter W1 Score EMA(s-t)',
    'Twitter W1 Score SMA(l-t)',
    'Twitter W1 Score SMA(s-t)',
    'Twitter W1 Score WMA(l-t)',
    'Twitter W1 Score WMA(s-t)',
    'VIm',
    'Volume',
    'W1 Score']



    X_test = PredictDF[feature_names_JMJ]
    y_test = PredictDF['Actual_Label']

    scaler = MinMaxScaler()
    X_train_scaled = scaler.fit_transform(X_test)
    X_test_scaled = scaler.transform(X_test)

#     predict_y1= np.argmax(stacking_classifier.predict_proba(X_test_scaled),axis=0)
    predict_y=  Model2.predict_proba(X_test_scaled)[:,1]

    #PredictDF['Predicted_Label']= predict_y

#     print(classification_report(y_test, predict_y))   

    testy_predThreshold = np.where(predict_y > UpperThreshold, 1,np.where(predict_y < LowerThresshold, 0,2))
    testy_predThreshold = testy_predThreshold.squeeze()
    dataset = pd.DataFrame({'Y_TEST': y_test, 'Y_PREDICTED': testy_predThreshold}, columns=['Y_TEST', 'Y_PREDICTED'])
    dataset2 = dataset.drop(dataset[dataset.Y_PREDICTED == 2].index)
   
    try:
#         LTSMaccuracy = accuracy_score(dataset2['Y_TEST'].values, dataset2['Y_PREDICTED'].values)
#         LTSM_RESULTS.append(LTSMaccuracy)
    
#         # Print classification report
        print(classification_report(dataset2['Y_TEST'].values, dataset2['Y_PREDICTED'].values))
        print(confusion_matrix(dataset2['Y_TEST'].values, dataset2['Y_PREDICTED'].values))
    except:
        print("No values")
    


              precision    recall  f1-score   support

         0.0       0.53      0.10      0.17       434
         1.0       0.57      0.93      0.71       562

    accuracy                           0.57       996
   macro avg       0.55      0.52      0.44       996
weighted avg       0.55      0.57      0.47       996

[[ 43 391]
 [ 38 524]]


In [3]:
INDICATORS = ['RSI', 'MACD', 'STOCH','ADL', 'ATR', 'MOM', 'MFI', 'ROC', 'OBV', 'CCI', 'EMV', 'VORTEX']

testdata2022 ='assets/TEST_MergeJMJPCHourly2019202_NewsGoogleApi_ActualLabel.csv'

files= [testdata2022]

def binary(value):
      if value > 0:
        return 1
      else:
        return 0

def _exponential_smooth(data, alpha):
    """
    Function that exponentially smooths dataset so values are less 'rigid'
    :param alpha: weight factor to weight recent values more
    """
    
    return data.ewm(alpha=alpha).mean()

def _get_indicator_data(data):
    """
    Function that uses the finta API to calculate technical indicators used as the features
    :return:
    """

    for indicator in INDICATORS:
        ind_data = eval('TA.' + indicator + '(data)')
        if not isinstance(ind_data, pd.DataFrame):
            ind_data = ind_data.to_frame()
        data = data.merge(ind_data, left_index=True, right_index=True)
    data.rename(columns={"14 period EMV.": '14 period EMV'}, inplace=True)

    # Also calculate moving averages for features
    data['ema50'] = data['Close'] / data['Close'].ewm(50).mean()
    data['ema21'] = data['Close'] / data['Close'].ewm(21).mean()
    data['ema15'] = data['Close'] / data['Close'].ewm(14).mean()
    data['ema5'] = data['Close'] / data['Close'].ewm(5).mean()

    # Instead of using the actual volume value (which changes over time), we normalize it with a moving volume average
    data['normVol'] = data['Volume'] / data['Volume'].ewm(5).mean()

    return data

UpperThreshold = 0.55
LowerThresshold = 0.45

for file,x in zip(files, ['testdata2022']):
    PredictDF= pd.read_csv(file)
    prediction = (PredictDF.shift(-1)['Close'] >= PredictDF['Close'])
    prediction = prediction.iloc[:-1]
    PredictDF['Actual_Label'] = prediction.astype(int)
    PredictDF= PredictDF.dropna()
    
    Indicatordata = _exponential_smooth(PredictDF[['Close', 'Open','High','Low','Volume']], 0.65)

    Indicatordatafinal = _get_indicator_data(Indicatordata)

    Indicatordatafinal = Indicatordatafinal.drop(['Close', 'Open', 'High', 'Low', 'Volume'], axis = 1)
    PredictDF = pd.merge(PredictDF, Indicatordatafinal, left_index=True, right_index=True)
    PredictDF = PredictDF.drop(['time','hour','Open Time','_merge','Signal','Position', 'Signal35JMJ', 'Position35JMJ', 'Ignore'], axis = 1)

    
    
    
    
    columns = ['favorites', 'retweets', 'number_of_followers', 'following',
           'followers_following_ratio', '2x_retweets_+_favorites', 'polarity',
           'W1 Score', 'Bull_ratio', 'W Score With Bull Ratio', 'Open', 'High',
           'Low', 'Close', 'Volume', 'Quote Asset Volume', 'Number of Trades',
           'TB Base Volume', 'TB Quote Volume', '3MovingAverage', '5MovingAverage',
           'JMJ_3HMoving_averages', 'JMJ_5HMoving_averages', 
           '14 period RSI', 'MACD', 'SIGNAL', '14 period STOCH %K', 'MFV',
           '14 period ATR', 'MOM', '14 period MFI', 'ROC', 'OBV', '20 period CCI',
           '14 period EMV', 'VIm', 'VIp', 'ema50', 'ema21', 'ema15', 'ema5']




    for column in columns: 
        PredictDF['Binary{}'.format(column)]  = (PredictDF[column] - PredictDF[column].shift(1)).apply(binary)
    

    PredictDF = PredictDF.dropna()
    
    feature_names_JMJ = [
    '# of Hourly Trades SMA(l-t)',
    '# of Hourly Trades SMA(s-t)',
    '# of Hourly Trades WMA(l-t)',
    '14 period MFI',
    '14 period STOCH %K',
    '20 period CCI',
    '2x_retweets_+_favorites',
    '3MovingAverage',
    'BinaryJMJ_3HMoving_averages',
    'BinaryJMJ_5HMoving_averages',
    'BinaryMFV',
    'BinaryOpen',
    'BinaryQuote Asset Volume',
    'BinaryVolume',
    'Bitcoin_Google_Trend_Score',
    'BTC_Google_Trend_Score',
    'Bull_ratio',
    'ema5',
    'ema50',
    'favorites',
    'followers_following_ratio',
    'following',
    'Historically Optimal EMA(l-t)',
    'Historically Optimal SMA(s-t)',
    'Historically Optimal WMA(s-t)',
    'MFV',
    'Mkt Sentiment',
    'normVol',
    'Number of Trades',
    'OBV',
    'polarity',
    'Quote Asset Volume',
    'Quote Asset Volume SMA(l-t)',
    'Quote Asset Volume SMA(s-t)',
    'retweets',
    'ROC',
    'TB Base Volume',
    'TB Base Volume EMA(l-t)',
    'TB Base Volume EMA(s-t)',
    'TB Base Volume SMA(l-t)',
    'TB Base Volume WMA(l-t)',
    'TB Base Volume WMA(s-t)',
    'TB Quote Volume',
    'TB Quote Volume WMA(s-t)',
    'Twitter Hourly 2x Retweets + Favorites EMA(s-t)',
    'Twitter Hourly 2x Retweets + Favorites SMA(s-t)',
    'Twitter Hourly Bull Ratio EMA(s-t)',
    'Twitter Hourly Bull Ratio SMA(l-t)',
    'Twitter Hourly Bull Ratio SMA(s-t)',
    'Twitter Hourly Bull Ratio WMA(l-t)',
    'Twitter Hourly Bull Ratio WMA(s-t)',
    'Twitter Hourly Favorites EMA(s-t)',
    'Twitter Hourly Favorites WMA(l-t)',
    'Twitter Hourly Favorites WMA(s-t)',
    'Twitter Hourly Follower Exposure EMA(l-t)',
    'Twitter Hourly Follower Exposure EMA(s-t)',
    'Twitter Hourly Follower Exposure SMA(s-t)',
    'Twitter Hourly Follower Exposure WMA(l-t)' ,
    'Twitter Hourly Follower Exposure WMA(s-t)',
    'Twitter Hourly Following Exposure EMA(l-t)',
    'Twitter Hourly Following Exposure SMA(s-t)',
    'Twitter Hourly Following Exposure WMA(s-t)',
    'Twitter Hourly Polarity Score EMA(l-t)',
    'Twitter Hourly Polarity Score EMA(s-t)',
    'Twitter Hourly Polarity Score SMA(l-t)',
    'Twitter Hourly Polarity Score SMA(s-t)',
    'Twitter Hourly Polarity Score WMA(l-t)',
    'Twitter Hourly Polarity Score WMA(s-t)',
    'Twitter Hourly Retweets EMA(l-t)',
    'Twitter Hourly Retweets SMA(l-t)',
    'Twitter Hourly Retweets SMA(s-t)',
    'Twitter Hourly Retweets WMA(s-t)',
    'Twitter Hourly Weighted Bull Ratio EMA(l-t)',
    'Twitter Hourly Weighted Bull Ratio EMA(s-t)',
    'Twitter Hourly Weighted Bull Ratio SMA(l-t)',
    'Twitter Hourly Weighted Bull Ratio SMA(s-t)',
    'Twitter Hourly Weighted Bull Ratio WMA(l-t)',
    'Twitter Hourly Weighted Bull Ratio WMA(s-t)',
    'Twitter W1 Score EMA(l-t)',
    'Twitter W1 Score EMA(s-t)',
    'Twitter W1 Score SMA(l-t)',
    'Twitter W1 Score SMA(s-t)',
    'Twitter W1 Score WMA(l-t)',
    'Twitter W1 Score WMA(s-t)',
    'VIm',
    'Volume',
    'W1 Score']



    X_test = PredictDF[feature_names_JMJ]
    y_test = PredictDF['Actual_Label']

    scaler = MinMaxScaler()
    X_train_scaled = scaler.fit_transform(X_test)
    X_test_scaled = scaler.transform(X_test)

#     predict_y1= np.argmax(stacking_classifier.predict_proba(X_test_scaled),axis=0)
    predict_y=  Model2.predict_proba(X_test_scaled)[:,1]

    #PredictDF['Predicted_Label']= predict_y

#     print(classification_report(y_test, predict_y))   

    testy_predThreshold = np.where(predict_y > UpperThreshold, 1,np.where(predict_y < LowerThresshold, 0,2))
    testy_predThreshold = testy_predThreshold.squeeze()
    dataset = pd.DataFrame({'Y_TEST': y_test, 'Y_PREDICTED': testy_predThreshold}, columns=['Y_TEST', 'Y_PREDICTED'])
    dataset2 = dataset.drop(dataset[dataset.Y_PREDICTED == 2].index)
   
    try:
#         LTSMaccuracy = accuracy_score(dataset2['Y_TEST'].values, dataset2['Y_PREDICTED'].values)
#         LTSM_RESULTS.append(LTSMaccuracy)
    
#         # Print classification report
        print(classification_report(dataset2['Y_TEST'].values, dataset2['Y_PREDICTED'].values))
        print(confusion_matrix(dataset2['Y_TEST'].values, dataset2['Y_PREDICTED'].values))
    except:
        print("No values")
    

  

              precision    recall  f1-score   support

         0.0       0.58      0.41      0.48      1922
         1.0       0.56      0.71      0.62      1992

    accuracy                           0.56      3914
   macro avg       0.57      0.56      0.55      3914
weighted avg       0.57      0.56      0.55      3914

[[ 796 1126]
 [ 584 1408]]


### Exporting Predictions

In [5]:
INDICATORS = ['RSI', 'MACD', 'STOCH','ADL', 'ATR', 'MOM', 'MFI', 'ROC', 'OBV', 'CCI', 'EMV', 'VORTEX']



testdata2022 ='assets/TEST_MergeJMJPCHourly2019202_NewsGoogleApi_ActualLabel.csv'

files= [testdata2022]

def binary(value):
      if value > 0:
        return 1
      else:
        return 0

def _exponential_smooth(data, alpha):
    """
    Function that exponentially smooths dataset so values are less 'rigid'
    :param alpha: weight factor to weight recent values more
    """
    
    return data.ewm(alpha=alpha).mean()

def _get_indicator_data(data):
    """
    Function that uses the finta API to calculate technical indicators used as the features
    :return:
    """

    for indicator in INDICATORS:
        ind_data = eval('TA.' + indicator + '(data)')
        if not isinstance(ind_data, pd.DataFrame):
            ind_data = ind_data.to_frame()
        data = data.merge(ind_data, left_index=True, right_index=True)
    data.rename(columns={"14 period EMV.": '14 period EMV'}, inplace=True)

    # Also calculate moving averages for features
    data['ema50'] = data['Close'] / data['Close'].ewm(50).mean()
    data['ema21'] = data['Close'] / data['Close'].ewm(21).mean()
    data['ema15'] = data['Close'] / data['Close'].ewm(14).mean()
    data['ema5'] = data['Close'] / data['Close'].ewm(5).mean()

    # Instead of using the actual volume value (which changes over time), we normalize it with a moving volume average
    data['normVol'] = data['Volume'] / data['Volume'].ewm(5).mean()

    return data

UpperThreshold = 0.5
LowerThresshold = 0.5

for file,x in zip(files, ['testdata2022']):
    PredictDF= pd.read_csv(file)
    prediction = (PredictDF.shift(-1)['Close'] >= PredictDF['Close'])
    prediction = prediction.iloc[:-1]
    PredictDF['Actual_Label'] = prediction.astype(int)
    PredictDF= PredictDF.dropna()
    
    Indicatordata = _exponential_smooth(PredictDF[['Close', 'Open','High','Low','Volume']], 0.65)

    Indicatordatafinal = _get_indicator_data(Indicatordata)

    Indicatordatafinal = Indicatordatafinal.drop(['Close', 'Open', 'High', 'Low', 'Volume'], axis = 1)
    PredictDF = pd.merge(PredictDF, Indicatordatafinal, left_index=True, right_index=True)
    PredictDF = PredictDF.drop(['time','hour','Open Time','_merge','Signal','Position', 'Signal35JMJ', 'Position35JMJ', 'Ignore'], axis = 1)

    
    
    
    
    columns = ['favorites', 'retweets', 'number_of_followers', 'following',
           'followers_following_ratio', '2x_retweets_+_favorites', 'polarity',
           'W1 Score', 'Bull_ratio', 'W Score With Bull Ratio', 'Open', 'High',
           'Low', 'Close', 'Volume', 'Quote Asset Volume', 'Number of Trades',
           'TB Base Volume', 'TB Quote Volume', '3MovingAverage', '5MovingAverage',
           'JMJ_3HMoving_averages', 'JMJ_5HMoving_averages', 
           '14 period RSI', 'MACD', 'SIGNAL', '14 period STOCH %K', 'MFV',
           '14 period ATR', 'MOM', '14 period MFI', 'ROC', 'OBV', '20 period CCI',
           '14 period EMV', 'VIm', 'VIp', 'ema50', 'ema21', 'ema15', 'ema5']




    for column in columns: 
        PredictDF['Binary{}'.format(column)]  = (PredictDF[column] - PredictDF[column].shift(1)).apply(binary)
    

    PredictDF = PredictDF.dropna()
    
    feature_names_JMJ = [
    '# of Hourly Trades SMA(l-t)',
    '# of Hourly Trades SMA(s-t)',
    '# of Hourly Trades WMA(l-t)',
    '14 period MFI',
    '14 period STOCH %K',
    '20 period CCI',
    '2x_retweets_+_favorites',
    '3MovingAverage',
    'BinaryJMJ_3HMoving_averages',
    'BinaryJMJ_5HMoving_averages',
    'BinaryMFV',
    'BinaryOpen',
    'BinaryQuote Asset Volume',
    'BinaryVolume',
    'Bitcoin_Google_Trend_Score',
    'BTC_Google_Trend_Score',
    'Bull_ratio',
    'ema5',
    'ema50',
    'favorites',
    'followers_following_ratio',
    'following',
    'Historically Optimal EMA(l-t)',
    'Historically Optimal SMA(s-t)',
    'Historically Optimal WMA(s-t)',
    'MFV',
    'Mkt Sentiment',
    'normVol',
    'Number of Trades',
    'OBV',
    'polarity',
    'Quote Asset Volume',
    'Quote Asset Volume SMA(l-t)',
    'Quote Asset Volume SMA(s-t)',
    'retweets',
    'ROC',
    'TB Base Volume',
    'TB Base Volume EMA(l-t)',
    'TB Base Volume EMA(s-t)',
    'TB Base Volume SMA(l-t)',
    'TB Base Volume WMA(l-t)',
    'TB Base Volume WMA(s-t)',
    'TB Quote Volume',
    'TB Quote Volume WMA(s-t)',
    'Twitter Hourly 2x Retweets + Favorites EMA(s-t)',
    'Twitter Hourly 2x Retweets + Favorites SMA(s-t)',
    'Twitter Hourly Bull Ratio EMA(s-t)',
    'Twitter Hourly Bull Ratio SMA(l-t)',
    'Twitter Hourly Bull Ratio SMA(s-t)',
    'Twitter Hourly Bull Ratio WMA(l-t)',
    'Twitter Hourly Bull Ratio WMA(s-t)',
    'Twitter Hourly Favorites EMA(s-t)',
    'Twitter Hourly Favorites WMA(l-t)',
    'Twitter Hourly Favorites WMA(s-t)',
    'Twitter Hourly Follower Exposure EMA(l-t)',
    'Twitter Hourly Follower Exposure EMA(s-t)',
    'Twitter Hourly Follower Exposure SMA(s-t)',
    'Twitter Hourly Follower Exposure WMA(l-t)' ,
    'Twitter Hourly Follower Exposure WMA(s-t)',
    'Twitter Hourly Following Exposure EMA(l-t)',
    'Twitter Hourly Following Exposure SMA(s-t)',
    'Twitter Hourly Following Exposure WMA(s-t)',
    'Twitter Hourly Polarity Score EMA(l-t)',
    'Twitter Hourly Polarity Score EMA(s-t)',
    'Twitter Hourly Polarity Score SMA(l-t)',
    'Twitter Hourly Polarity Score SMA(s-t)',
    'Twitter Hourly Polarity Score WMA(l-t)',
    'Twitter Hourly Polarity Score WMA(s-t)',
    'Twitter Hourly Retweets EMA(l-t)',
    'Twitter Hourly Retweets SMA(l-t)',
    'Twitter Hourly Retweets SMA(s-t)',
    'Twitter Hourly Retweets WMA(s-t)',
    'Twitter Hourly Weighted Bull Ratio EMA(l-t)',
    'Twitter Hourly Weighted Bull Ratio EMA(s-t)',
    'Twitter Hourly Weighted Bull Ratio SMA(l-t)',
    'Twitter Hourly Weighted Bull Ratio SMA(s-t)',
    'Twitter Hourly Weighted Bull Ratio WMA(l-t)',
    'Twitter Hourly Weighted Bull Ratio WMA(s-t)',
    'Twitter W1 Score EMA(l-t)',
    'Twitter W1 Score EMA(s-t)',
    'Twitter W1 Score SMA(l-t)',
    'Twitter W1 Score SMA(s-t)',
    'Twitter W1 Score WMA(l-t)',
    'Twitter W1 Score WMA(s-t)',
    'VIm',
    'Volume',
    'W1 Score']


    X_test = PredictDF[feature_names_JMJ]
    y_test = PredictDF['Actual_Label']

    scaler = MinMaxScaler()
    X_train_scaled = scaler.fit_transform(X_test)
    X_test_scaled = scaler.transform(X_test)

#     predict_y1= np.argmax(stacking_classifier.predict_proba(X_test_scaled),axis=0)
    #predict_y=  Model2.predict_proba(X_test_scaled)[:,1]

    PredictDF['Prediction']= Model2.predict_proba(X_test_scaled)[:,1]


In [6]:
Model2

StackingClassifier(cv=5,
                   estimators=[('lr',
                                LogisticRegression(C=0.01, max_iter=1000)),
                               ('abc',
                                AdaBoostClassifier(learning_rate=0.001,
                                                   n_estimators=20)),
                               ('lda', LinearDiscriminantAnalysis()),
                               ('GBC',
                                GradientBoostingClassifier(learning_rate=0.01))],
                   final_estimator=LogisticRegression(C=0.01, max_iter=1000),
                   passthrough=True, stack_method='predict_proba')

In [7]:
PredictDF

favorites  retweets  number_of_followers    following  \
14     9.797619  0.745238         16514.150000  1498.323810   
15    39.326190  3.390476         16099.623810  1672.185714   
16     5.369048  0.550000         11057.442857  1828.021429   
17    14.495238  1.890476         25029.171429  1882.980952   
18    13.329545  1.281818         22107.427273  2018.227273   
...         ...       ...                  ...          ...   
7263  10.030952  2.557143         12561.900000   906.071429   
7264   9.507143  3.530952         10695.850000   805.514286   
7265  17.957143  4.402381         22596.990476   846.535714   
7266   4.788095  1.021429         21378.707143   730.971429   
7267  10.600000  1.764286         15495.466667   663.738095   

      followers_following_ratio  2x_retweets_+_favorites  polarity  W1 Score  \
14                    63.299076                11.288095  0.113520  0.128081   
15                   110.504489                46.107143  0.087327  0.253405   
16                   133.780075                 6.469048  0.093584  0.222649   
17                   662.631886                18.276190  0.090903  0.173174   
18                   479.791704                15.893182  0.095774  0.263250   
...                         ...                      ...       ...       ...   
7263                 406.088057                15.145238  0.094200  0.365355   
7264                 264.229345                16.569048  0.075146  0.250643   
7265                 217.348828                26.761905  0.104180  0.403896   
7266                 172.471290                 6.830952  0.097456  0.310215   
7267                 170.085989                14.128571  0.078864  0.206627   

      Bull_ratio  W Score With Bull Ratio  ...  BinaryOBV  \
14      4.256410                 0.545164  ...          0   
15      2.500000                 0.633513  ...          0   
16      2.450000                 0.545491  ...          0   
17      2.500000                 0.432936  ...          1   
18      3.395833                 0.893952  ...          1   
...          ...                      ...  ...        ...   
7263    3.066667                 1.120422  ...          1   
7264    2.518519                 0.631248  ...          0   
7265    2.941176                 1.187930  ...          0   
7266    4.176471                 1.295602  ...          1   
7267    3.250000                 0.671536  ...          0   

      Binary20 period CCI  Binary14 period EMV  BinaryVIm  BinaryVIp  \
14                      1                    0          0          0   
15                      0                    0          1          0   
16                      1                    0          1          0   
17                      1                    1          0          1   
18                      1                    1          0          1   
...                   ...                  ...        ...        ...   
7263                    1                    1          0          1   
7264                    1                    1          0          1   
7265                    0                    0          1          1   
7266                    1                    0          1          0   
7267                    1                    0          1          0   

      Binaryema50  Binaryema21  Binaryema15  Binaryema5  Prediction  
14              0            0            0           0    0.635768  
15              0            0            0           0    0.639907  
16              0            0            0           1    0.626424  
17              1            1            1           1    0.595970  
18              1            1            1           1    0.462729  
...           ...          ...          ...         ...         ...  
7263            1            1            1           1    0.644039  
7264            0            0            0           1    0.626921  
7265            0            1            1           1    0

In [9]:
PredictDF.to_csv('predictions/PriotyFeatures_StackingPrediction_LR_ABC_LDA_GBC_to_LR.csv')